##### Bibliotecas

In [93]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import re
import time

##### Navegador

In [94]:
nav = webdriver.Chrome()

##### Base de dados

In [95]:
tabela_produtos = pd.read_excel('buscas.xlsx')
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


##### Google Shopping

- Variáveis

In [96]:
produto = "iphone 12 64 gb"
termos_banidos = 'mini watch'
preco_minimo = 3000
preco_maximo = 4500

- Função

In [97]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
  tem_termos_banidos = False
  for palavra in termos_banidos.split(' '):
    if palavra in nome:
      tem_termos_banidos = True
  return tem_termos_banidos

def verificar_tem_todos_termos_produtos(lista_termos_nome_produto, nome):
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
      if palavra not in nome:
        tem_todos_termos_produtos = False   
    return tem_todos_termos_produtos

def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):

  produto = produto.lower()
  termos_banidos = termos_banidos.lower()
  lista_termos_banidos = termos_banidos.split(' ')
  lista_termos_nome_produto = produto.split(' ')
  lista_ofertas = []
  
  preco_minimo = float(preco_minimo)
  preco_maximo = float(preco_maximo)

  nav.get('https://www.google.com.br/')

  nav.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER)

  # Aba Shopping
  elementos = nav.find_element(By.CLASS_NAME, 'UqcIvb').click()

  # Preços
  lista_resultados = nav.find_elements(By.CLASS_NAME, 'KZmu8e')

  for resultado in lista_resultados:
    nome = resultado.find_element(By.CLASS_NAME, 'sh-np__product-title').text
    nome = nome.lower()

    tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
    tem_todos_termos_produtos = verificar_tem_todos_termos_produtos(lista_termos_nome_produto, nome)

    if not tem_termos_banidos and tem_todos_termos_produtos:    
      preco = resultado.find_element(By.CLASS_NAME, 'hn9kf').text
      preco = re.search(r'\d+\.?\d+', preco).group()
      preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
      preco = float(preco)

      if preco_minimo <= preco <= preco_maximo:
        elemento_referencia = resultado.find_element(By.CLASS_NAME, 'SirUVb')
        elemento_pai = elemento_referencia.find_element(By.XPATH, './..')
        link = elemento_pai.get_attribute('href')
        lista_ofertas.append((nome, preco, link))
  return lista_ofertas


def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):

  produto = produto.lower()
  termos_banidos = termos_banidos.lower()
  lista_termos_banidos = termos_banidos.split(' ')
  lista_termos_nome_produto = produto.split(' ')
  lista_ofertas = []
  preco_minimo = float(preco_minimo)
  preco_maximo = float(preco_maximo)

  nav.get('https://www.buscape.com.br/')
  nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)

  time.sleep(5)
    
  lista_resultados = nav.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')

  for resultado in lista_resultados:
    preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
    nome = resultado.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text
    nome = nome.lower()
    link = resultado.get_attribute('href')

    tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
    tem_todos_termos_produtos = verificar_tem_todos_termos_produtos(lista_termos_nome_produto, nome)

    if not tem_termos_banidos and tem_todos_termos_produtos:  
      preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
      preco = float(preco)
      if preco_minimo <= preco <= preco_maximo:
        lista_ofertas.append((nome, preco, link))

  return lista_ofertas

In [98]:
lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
print(lista_ofertas_google_shopping)

lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
print(lista_ofertas_buscape)

[('iphone 12 apple 64gb preto tela 6,1 12mp ios, 64gb, preto', 3749.0, 'https://www.google.com.br/aclk?sa=l&ai=DChcSEwj7nYj4q7uAAxVBw5EKHQG_BacYABABGgJjZQ&sig=AOD64_1tvpoU3fT9ybZtm7dJGIg7a0offw&ctype=46&q=&ved=0ahUKEwin94P4q7uAAxXERLgEHT4FDXsQqygIsQw&adurl='), ('smartphone apple iphone 12, 64gb, preto, 5g, 6.1" super retina xdr oled, câmera dupla 12mp, selfie 12mp, ios 15', 3599.0, 'https://www.google.com.br/aclk?sa=l&ai=DChcSEwj7nYj4q7uAAxVBw5EKHQG_BacYABADGgJjZQ&sig=AOD64_3JvE_Ddt8XQdPpsL6HmQt6wgvW0Q&ctype=5&q=&ved=0ahUKEwin94P4q7uAAxXERLgEHT4FDXsQww8ItQw&adurl='), ('iphone 12 apple 64gb preto tela de 6,1, câmera dupla de 12mp, ios', 3749.0, 'https://www.google.com.br/aclk?sa=l&ai=DChcSEwj7nYj4q7uAAxVBw5EKHQG_BacYABAFGgJjZQ&sig=AOD64_17y_zd0I6_j1MI3SJBdTA7Lda7KA&ctype=5&q=&ved=0ahUKEwin94P4q7uAAxXERLgEHT4FDXsQww8IuQw&adurl='), ('iphone 12 64gb roxo tela 6,1 4g câmera traseira 12mp vitrine', 3299.0, 'https://www.google.com.br/aclk?sa=l&ai=DChcSEwj7nYj4q7uAAxVBw5EKHQG_BacYABAHGgJjZQ&si